# Individual Project

## Data Analysis and Visualization of Large Wildfire Spread in Portugal

In [4]:
import pandas as pd
from simpledbf import Dbf5
import numpy as np

In [43]:
firespread = pd.read_excel("PT-FireSprd_L3_FireBehavior.xlsx")
firespread

,fid,fname,burn_perio,year,sdate,edate,qc,inidoy,enddoy,duration,area,growth_rat,ros,spdir,max_ros,FRE,FRE_perc,FRE_flux
0,29,Abrantes_09082017,1,2017,2017-08-09 18:00,2017-08-09 23:00,3,221.750000,221.958333,5.0,500.00,100.000160,1199.623292,155.744885,1199.623292,-1.000000,60.0,-1.000000
1,29,Abrantes_09082017,2,2017,2017-08-09 23:00,2017-08-10 03:00,3,221.958333,222.125000,4.0,195.75,48.937402,388.377944,181.152080,393.534571,-1.000000,12.5,-1.000000
2,29,Abrantes_09082017,3,2017,na,2017-08-10 12:00,3,-1.000000,222.500000,-1.0,191.75,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.0,-1.000000
3,29,Abrantes_09082017,4,2017,2017-08-10 12:00,2017-08-10 14:30,3,222.500000,222.604167,2.5,1184.25,473.698484,1088.042456,238.370571,1088.042456,11.371950,100.0,3.841064
4,29,Abrantes_09082017,5,2017,2017-08-10 14:30,2017-08-10 22:00,3,222.604167,222.916667,7.5,1552.50,207.000000,580.932693,170.189772,585.006347,40.084560,100.0,3.442582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,59,ViladeRei_20072019,1,2019,2019-07-20 14:30,2019-07-20 23:00,3,201.604167,201.958333,8.5,3141.75,369.618343,2258.715621,125.157120,5354.374991,86.076792,100.0,3.223261
266,59,ViladeRei_20072019,2,2019,2019-07-20 18:00,2019-07-21 03:00,3,201.750000,202.125000,9.0,3015.25,335.027778,1040.574990,282.638762,5094.328506,77.008247,100.0,2.837732
267,59,ViladeRei_20072019,3,2019,2019-07-21 14:30,2019-07-21 23:30,3,202.604167,202.979167,9.0,1984.00,220.444444,1265.237483,128.338786,3668.894012,72.506970,100.0,4.060650
268,59,ViladeRei_20072019,4,2019,2019-07-22 14:00,2019-07-22 18:00,3,203.583333,203.750000,4.0,878.00,219.499561,1251.174723,112.528592,1584.593086,26.322301,100.0,7.494960


In [44]:
firespread.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   fid         270 non-null    int64  
 1   fname       270 non-null    object 
 2   burn_perio  270 non-null    int64  
 3   year        270 non-null    int64  
 4   sdate       270 non-null    object 
 5   edate       270 non-null    object 
 6   qc          270 non-null    int64  
 7   inidoy      270 non-null    float64
 8   enddoy      270 non-null    float64
 9   duration    270 non-null    float64
 10  area        270 non-null    float64
 11  growth_rat  270 non-null    float64
 12  ros         270 non-null    float64
 13  spdir       270 non-null    float64
 14  max_ros     270 non-null    float64
 15  FRE         270 non-null    float64
 16  FRE_perc    270 non-null    float64
 17  FRE_flux    270 non-null    float64
dtypes: float64(11), int64(4), object(3)
memory usage: 38.1+ KB


In [45]:
firespread.columns

Index(['fid', 'fname', 'burn_perio', 'year', 'sdate', 'edate', 'qc', 'inidoy',
       'enddoy', 'duration', 'area', 'growth_rat', 'ros', 'spdir', 'max_ros',
       'FRE', 'FRE_perc', 'FRE_flux'],
      dtype='object')

### To do:

- group fires by id

df.groupby('A').B.agg(['min', 'max'])
df.groupby('A').agg({'B': ['min', 'max'], 'C': 'sum'})
df.groupby(['A','othercolumn']).agg({'B': ['min', 'max'], 'C': 'sum'})

- use datetime to extract month from the dates and make column month
- if possible make correlation with weather like wind/air humidity/etc
